# Convert model 

In [2]:
import torch, json, os
from torchvision import models

root_model = "/home/pi5/TrafficSign/WeightClassify"
root_data = "/home/pi5/TrafficSign/Dataset/Classify"

with open(os.path.join(root_data, "label2idx.json")) as f:
    label2idx = json.load(f)

num_classes = len(label2idx)
print("🔹 Num classes:", num_classes)


🔹 Num classes: 91


## Export sang ONNX (FP32)

In [ ]:
import os, torch

def export_onnx(model, model_name, input_size=(3, 64, 64)):
    model.eval()
    dummy = torch.randn(1, *input_size)

    # === Tạo thư mục export riêng ===
    export_dir = f"/home/pi5/TrafficSign/convert/classify_model/export_onnx/{model_name}"
    os.makedirs(export_dir, exist_ok=True)

    # === Đường dẫn lưu file ===
    onnx_path = os.path.join(export_dir, f"{model_name}_fp32.onnx")

    # === Export FP32 ===
    torch.onnx.export(
        model, dummy, onnx_path,
        input_names=['input'], output_names=['output'],
        opset_version=18
    )
    print(f"✅ Exported FP32: {onnx_path}")

    # === Convert sang FP16 ===
    import onnx
    from onnxconverter_common import float16

    onnx_fp32 = onnx.load(onnx_path)
    onnx.save(onnx_fp32, onnx_path)


In [ ]:
# === ResNet18 ===
model = models.resnet18()
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(f"{root_model}/resnet18_best.pth", map_location="cpu"))
export_onnx(model, "resnet18")

# === EfficientNetB0 ===
model = models.efficientnet_b0()
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)
model.load_state_dict(torch.load(f"{root_model}/efficientnetb0_best.pth", map_location="cpu"))
export_onnx(model, "efficientnetb0")

# === MobileNetV2 ===
model = models.mobilenet_v2()
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)
model.load_state_dict(torch.load(f"{root_model}/mobilenetv2_best.pth", map_location="cpu"))
export_onnx(model, "mobilenetv2")

# === ShuffleNetV2 ===
model = models.shufflenet_v2_x1_0()
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(f"{root_model}/shufflenetv2_x1_0_best.pth", map_location="cpu"))
export_onnx(model, "shufflenetv2_x1_0")


## Export sang OpenVINO

In [ ]:
# import openvino as ov

# onnx_path = "/home/pi5/TrafficSign/convert/classify_model/export_onnx/resnet18/resnet18_fp32.onnx"
# output_dir = "/home/pi5/TrafficSign/convert/classify_model/export_openvino/resnet18"

# core = ov.Core()

# # Convert model từ ONNX sang OpenVINO IR
# model = ov.convert_model(onnx_path, framework="onnx")

# # Serialize ra file .xml + .bin
# ov.save_model(model, f"{output_dir}/resnet18.xml")


## Export sang NCNN

In [ ]:
import os
import subprocess
from datetime import datetime

def export_to_ncnn(model_name, 
                   onnx_root="/kaggle/working/export_onnx", 
                   out_root="/kaggle/working/export_ncnn",
                   input_shape="[1,3,64,64]"):
    """
    Export ONE model (model_name) from ONNX → NCNN using pnnx converter.
    """
    model_dir = os.path.join(out_root, model_name)
    os.makedirs(model_dir, exist_ok=True)

    onnx_path = os.path.join(onnx_root, model_name, f"{model_name}_fp16.onnx")
    if not os.path.exists(onnx_path):
        print(f"⚠️ Model not found: {onnx_path}")
        return None

    print(f"\n🚀 Exporting {model_name} to NCNN...")
    start = datetime.now()
    cmd = [
        "pnnx", onnx_path,
        f"inputshape={input_shape}f16",
        "device=cpu",
        f"outputdir={model_dir}"
    ]

    try:
        subprocess.run(cmd, check=True)
        elapsed = (datetime.now() - start).total_seconds()
        print(f"✅ Done {model_name} in {elapsed:.2f}s → {model_dir}")
        return {"model": model_name, "status": "success", "time_s": elapsed}
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed {model_name}: {e}")
        return {"model": model_name, "status": "failed", "time_s": None}

In [ ]:
export_to_ncnn("resnet18")

In [ ]:
export_to_ncnn("mobilenetv2")

In [ ]:
export_to_ncnn("efficientnetb0")

In [ ]:
export_to_ncnn("shufflenetv2_x1_0")

# Inference

## Inference ONNX

## Inference OpenVINO

In [3]:
import os, time, json
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from openvino.runtime import Core
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluate_openvino_model(xml_path, idx2label_path, data_dir = "/home/pi5/TrafficSign/Dataset/Classify/test"
, img_size=64, n_samples=1000):
    """
    Benchmark OpenVINO model (FP32) on CPU to match PyTorch evaluation settings.
    """
    # ==== Chuẩn bị dữ liệu ====
    idx2label = json.load(open(idx2label_path))
    label2idx = {v: k for k, v in idx2label.items()}

    # Lấy ảnh mẫu cố định
    samples = []
    for root, _, files in os.walk(data_dir):
        imgs = [os.path.join(root, f) for f in files if f.lower().endswith(('.jpg', '.png'))]
        for f in imgs:
            samples.append((f, os.path.basename(root)))

    if n_samples:
        samples = samples[:n_samples]

    # ==== Tiền xử lý ảnh (giống PyTorch ImageNet) ====
    def preprocess_image(path):
        img = cv2.imread(path)
        img = cv2.resize(img, (img_size, img_size))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0
        mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
        img = (img - mean) / std
        img = img.transpose(2, 0, 1)[None, ...]  # NCHW
        return img

    # ==== Khởi tạo OpenVINO ====
    core = Core()
    bin_path = xml_path.replace(".xml", ".bin")
    model = core.read_model(xml_path, bin_path)
    compiled = core.compile_model(model, "CPU")

    input_tensor = compiled.inputs[0]
    output_tensor = compiled.outputs[0]

    preds_all, labels_all = [], []
    total_time = 0.0

    # ==== Inference loop ====
    for img_path, true_label in tqdm(samples, desc=f"Evaluating {os.path.basename(xml_path)}"):
        img = preprocess_image(img_path)

        start = time.perf_counter()
        result = compiled([img])[output_tensor]
        total_time += time.perf_counter() - start

        pred_label = str(int(np.argmax(result)))
        preds_all.append(pred_label)
        labels_all.append(label2idx[true_label])

    # ==== Metrics ====
    preds_int = list(map(int, preds_all))
    labels_int = list(map(int, labels_all))

    acc = accuracy_score(labels_int, preds_int)
    prec = precision_score(labels_int, preds_int, average="macro", zero_division=0)
    rec = recall_score(labels_int, preds_int, average="macro", zero_division=0)
    f1 = f1_score(labels_int, preds_int, average="macro", zero_division=0)
    fps = len(samples) / total_time if total_time > 0 else 0

    return {
        "Model": os.path.basename(xml_path).replace(".xml", ""),
        "Device": "CPU",
        "Precision": "FP32",
        "Accuracy": round(acc, 4),
        "PrecisionScore": round(prec, 4),
        "Recall": round(rec, 4),
        "F1": round(f1, 4),
        "FPS": round(fps, 2),
        "Time_per_img(s)": round(total_time / len(samples), 4),
        "Samples": len(samples)
    }


In [5]:
# === Đường dẫn ===
base_dir = "/home/pi5/TrafficSign/convert/classify_model/export_openvino"
data_dir = "/home/pi5/TrafficSign/Dataset/Classify/test"
idx2label_path = "/home/pi5/TrafficSign/Dataset/Classify/idx2label.json"

# === Danh sách model ===
models = ["resnet18","mobilenetv2","efficientnetb0", "shufflenetv2_x1_0"]


results = []
for name in models:
    xml_path = os.path.join(base_dir, name, f"{name}_fp32.xml")
    if not os.path.exists(xml_path):
        print(f"⚠️ Missing: {xml_path}")
        continue
    res = evaluate_openvino_model(xml_path, idx2label_path, data_dir, img_size=64)
    results.append(res)

# === Kết quả ===
df_ov = pd.DataFrame(results)
display(df_ov.style.hide(axis="index").set_caption("OpenVINO FP32 (CPU) Inference Benchmark"))


Evaluating shufflenetv2_x1_0_fp32.xml: 100%|██████████| 1000/1000 [00:03<00:00, 251.94it/s]


Model,Device,Precision,Accuracy,PrecisionScore,Recall,F1,FPS,Time_per_img(s),Samples
resnet18_fp32,CPU,FP32,0.979000,0.443600,0.428400,0.435700,200.480000,0.005000,1000
mobilenetv2_fp32,CPU,FP32,0.980000,0.443800,0.425800,0.434400,334.940000,0.003000,1000
efficientnetb0_fp32,CPU,FP32,0.874000,0.294000,0.226100,0.248300,142.510000,0.007000,1000
shufflenetv2_x1_0_fp32,CPU,FP32,0.884000,0.330300,0.267600,0.290800,285.750000,0.003500,1000


In [6]:
models2 = ["efficientnetb0", "shufflenetv2_x1_0"]
results = []
for name in models2:
    xml_path = os.path.join(base_dir, name, f"{name}_fp32.xml")
    if not os.path.exists(xml_path):
        print(f"⚠️ Missing: {xml_path}")
        continue
    res = evaluate_openvino_model(xml_path, data_dir, idx2label_path, img_size=64)
    results.append(res)


Evaluating efficientnetb0_fp32.xml:  63%|██████▎   | 5854/9226 [01:30<00:51, 64.96it/s]  


KeyboardInterrupt: 

In [ ]:
df_ov = pd.DataFrame(results)
display(df_ov.style.hide(axis="index").set_caption("OpenVINO FP32 (CPU) Inference Benchmark"))

## Inference NCNN